In [ ]:
import importlib
import logging
log = logging.getLogger()
from scipy.signal import find_peaks, peak_widths
from scipy.signal import lti, step
import numpy as np
import control
from matplotlib import pyplot as plt

import pulse_transitions
importlib.reload(pulse_transitions)
from pulse_transitions import transient_response, GroupStrategy, CrossingDetectionSettings, midcross, overshoot, undershoot
import photoreceiver_analysis
from pulse_transitions.transient_response import Edge, detect_edges, detect_thresholds
#, _interpolate_crossing, _find_peaks_and_types
from photoreceiver_analysis.transientResponsePlot import plot_transient, format_fig
from photoreceiver_analysis.plotTools import make_engineering_note_labels

# Outline
+ Generate some signals
+ Manually calculate the signal levels, overshoot, and rise time
+ Check against the control library
+ Add noise, show it failing

# Algorithms
## Reference Levels: Find high and low levels
    + Use different algorithms with different requirements.
        + Mean of endpoints: endpoints must be the flat values
        + Histogram of average w/ peak fitting
        + Sections with a low derivative: Needs a minimum length relative to the entire length.
        + Manual entry

## Rise/Fall Time
+ Denoise the signal and take either the first, last, mean, or median crossing time

## Over/undershoot
+ Take the min and max values within the time of the edge and the settling time

In [ ]:
def make_lti_step_response(x, damping=0.25, noise_amplitude=0.02):
    # Damping ratio < 1 for overshoot
    omega = 2 * np.pi * 1  # Natural frequency (1 Hz)
    system = lti([omega**2], [1, 2*damping*omega, omega**2])
    
    # Step response only for t >= 0
    _, yp = step(system, T=x[x >= 0])
    
    # Add Gaussian noise to the response
    yp_noisy = yp + np.random.normal(0, noise_amplitude, size=yp.shape)
    
    # Prepend zeros for t < 0
    y = [0] * len(x[x < 0])
    y.extend(yp_noisy)
    y = np.array(y)
    return y

def plot_step(x, y, settings=None):
    # Transient analysis
    thresholds = transient_response.detect_thresholds(x, y)
    print(thresholds)
    if settings is None:
        settings = CrossingDetectionSettings()
    edges = detect_edges(x, y, thresholds, settings=settings)
    print(edges)
    # Plot
    fig, ax = plot_transient(x, y, edges, thresholds=thresholds, xscale=1, unit='s')
    
    # Format ticks with engineering notation
    ticks = make_engineering_note_labels(unit='s', ticks=ax.get_xticks(), start=0)
    ax.set_xticks(ticks[0])
    ax.set_xticklabels(ticks[1])
    
    ticks = make_engineering_note_labels(unit='V', ticks=ax.get_yticks(), start=0)
    ax.set_yticks(ticks[0])
    ax.set_yticklabels(ticks[1])
    
    # Optional: Apply any additional formatting
    # fig = format_fig(fig)
    
    return fig, ax

In [ ]:
y = np.concatenate([
    [0]*500,
    np.linspace(0.0, 1.0, 100),
    [1]*500,
    #np.linspace(1.0, 0.0, 500),
    #[0]*500,
    #np.linspace(0.0, 1.0, 500),
    #[1]*500,
    #np.linspace(1.0, 0.0, 500)
])

x = np.linspace(0, 10, len(y))


plt.plot(x, y)
plt.plot(x, np.gradient(y, x))
config = CrossingDetectionSettings(
    window=0)

edges = detect_edges(x, y, thresholds=(0.2, 0.8), settings=config)
print(edges)

In [ ]:
x = np.linspace(-2, 5, 1000)
y = -make_lti_step_response(x, damping=0.25, noise_amplitude=0.0)
mid = midcross(y, t=x)

print(overshoot(y, sigma_smooth=0))
print(undershoot(y, sigma_smooth=0))

plt.plot(x, y)
plt.plot([mid]*2, [min(y), max(y)])

In [ ]:
signal = list(y) + list(-y) + list(y) + list(-y) + list(y) + list(-y) + list(y) + list(-y)
xp = np.array(list(range(len(signal))))

settings = CrossingDetectionSettings(window=100)
plot_step(xp, signal, settings=settings)

signal = np.sin(xp/np.pi / 100)
plot_step(xp, signal, settings=settings)

In [ ]:
# Time vector
x = np.linspace(-1, 5, 1000)
for n in [0.02, 0.04, 0.06, 0.08, 0.1, 0.5]:
    y = make_lti_step_response(x, damping=0.25, noise_amplitude=n)
    fig, ax = plot_step(x, y)
    plt.show()
    print(control.step_info(sysdata=(y), T=x))

In [ ]:
def plot_edges(x, y, edges, levels):
    fig, ax = plt.subplots()
    ax.plot(x, y)
    ax.plot(x, [min(levels)]*len(x), 'k--')
    ax.plot(x, [max(levels)]*len(x), 'k--')

    ax2 = ax.twinx()
    ax2.plot(x, np.gradient(y, x), 'r')
    rising = list(filter(lambda x: x.type=='rise', edges))
    falling = list(filter(lambda x: x.type=='fall', edges))
    for pts in rising, falling:
        for pt in pts:
            pos = pt.start if pt.type=="rise" else pt.end
            ax.plot([pos]*2, levels, 'k--')
            ax.plot([pos]*2, [np.mean(levels)]*2, 'kx')
    return fig, ax
    
x = np.linspace(0, 10, 1000)
y = np.where(x > 5, 1.0, 0.0)
levels = detect_thresholds(x, y)
edges = detect_edges(x, y, levels)
plot_edges(x, y, edges, levels)
plt.show()

x = np.linspace(0, 10, 1000)
y = np.where(x > 5, -1.0, 0.0)
levels = detect_thresholds(x, y, "histogram", 0.1, 0.6)
edges = detect_edges(x, y, levels)
plot_edges(x, y, edges, levels)
plt.show()

In [ ]:
# System parameters: 2nd-order underdamped system
# Transfer function: H(s) = ω_n^2 / (s^2 + 2ζω_n s + ω_n^2)
damping = 0.5  # Damping ratio < 1 for overshoot
omega = 2 * np.pi * 1  # Natural frequency (1 Hz)
# Define the LTI system
system = lti([omega**2], [1, 2*damping*omega, omega**2])
x = np.linspace(-1, 5, 1000)
_, yp = step(system, T=x[x>=0])
y = [0]*len(x[x<0])
y.extend(yp)
y = np.array(y)
levels = transient_response.detect_thresholds(x, y)
edges = detect_edges(x, y, levels)

print(levels, edges)
plot_edges(x, y, edges, levels)
plt.show()
print(transient_response.detect_overshoot(x, y))

y = -y
levels = transient_response.detect_thresholds(x, y)
edges = detect_edges(x, y, levels)

print(levels, edges)
plot_edges(x, y, edges, levels)
plt.show()

print(transient_response.detect_overshoot(x, y))

In [ ]:
# System parameters: 2nd-order underdamped system
# Transfer function: H(s) = ω_n^2 / (s^2 + 2ζω_n s + ω_n^2)
damping = 0.1  # Damping ratio < 1 for overshoot
omega = 2 * np.pi * 1  # Natural frequency (1 Hz)
# Define the LTI system
system = lti([omega**2], [1, 2*damping*omega, omega**2])
x = np.linspace(-1, 10, 1000)
_, yp = step(system, T=x[x>=0])
y = [0]*len(x[x<0])
y.extend(yp)
y = np.array(y)
levels = transient_response.detect_thresholds(x, y)
edges = detect_edges(x, y, levels)

print(len(edges), edges)
for pt in edges:
    print(pt.dx)
fig, ax = plot_edges(x, y, edges, levels)
plt.show()
print(transient_response.detect_overshoot(x, y))

In [ ]:
# System parameters: 2nd-order underdamped system
# Transfer function: H(s) = ω_n^2 / (s^2 + 2ζω_n s + ω_n^2)
damping = 0.5  # Damping ratio < 1 for overshoot
omega = 2 * np.pi * 1  # Natural frequency (1 Hz)
# Define the LTI system
system = lti([omega**2], [1, 2*damping*omega, omega**2])
x = np.linspace(-1, 5, 1000)
_, yp = step(system, T=x[x>=0])
y = [0]*len(x[x<0])
y.extend(yp)
y = np.array(y)
levels = transient_response.detect_thresholds(x, y)
edges = detect_edges(x, y, levels)

fig, ax = plot_edges(x, y, edges, levels)
print(ax.get_xaxis().get_majorticklabels())
ticks = make_engineering_note_labels(unit='Hz', ticks=ax.get_xticks(), start=0)
print(ticks)

In [ ]:
damping = 0.5  # Damping ratio < 1 for overshoot
omega = 2 * np.pi * 1  # Natural frequency (1 Hz)
# Define the LTI system
system = lti([omega**2], [1, 2*damping*omega, omega**2])
x = np.linspace(-1, 5, 1000)
_, yp = step(system, T=x[x>=0])
y = [0]*len(x[x<0])
y.extend(yp)
y = np.array(y)
thresholds = transient_response.detect_thresholds(x, y)
edges = detect_edges(x, y, thresholds)

fig, ax = plot_transient(x, y, edges, thresholds=thresholds, xscale=1, unit='s')
ticks = make_engineering_note_labels(unit='s', ticks=ax.get_xticks(), start=0)
ax.set_xticklabels(ticks[1])

ticks = make_engineering_note_labels(unit='V', ticks=ax.get_yticks(), start=0)
ax.set_yticklabels(ticks[1])

# fig = format_fig(fig)
plt.show()

In [ ]:
# Define system
damping = 0.25  # Damping ratio < 1 for overshoot
omega = 2 * np.pi * 1  # Natural frequency (1 Hz)
system = lti([omega**2], [1, 2*damping*omega, omega**2])

# Time vector
x = np.linspace(-1, 5, 1000)

# Step response only for t >= 0
_, yp = step(system, T=x[x >= 0])

# Add Gaussian noise to the response
noise_amplitude = 0.02  # Adjust noise level here
yp_noisy = yp + np.random.normal(0, noise_amplitude, size=yp.shape)

# Prepend zeros for t < 0
y = [0] * len(x[x < 0])
y.extend(yp_noisy)
y = np.array(y)

# Transient analysis
thresholds = transient_response.detect_thresholds(x, y)
edges = detect_edges(x, y, thresholds)

# Plot
fig, ax = plot_transient(x, y, edges, thresholds=thresholds, xscale=1, unit='s')

# Format ticks with engineering notation
ticks = make_engineering_note_labels(unit='s', ticks=ax.get_xticks(), start=0)
ax.set_xticks(ticks[0])
ax.set_xticklabels(ticks[1])

ticks = make_engineering_note_labels(unit='V', ticks=ax.get_yticks(), start=0)
ax.set_yticks(ticks[0])
ax.set_yticklabels(ticks[1])

# Optional: Apply any additional formatting
# fig = format_fig(fig)

plt.show()

In [ ]:
control.step_info(sysdata=(list(-y)*2)/max(y), T=list(x)*2)

In [ ]:
yp = list(y)*3
xp = np.linspace(0, 10, len(yp))

thresholds = transient_response.detect_thresholds(xp, yp, method="histogram")
edges = detect_edges(xp, yp, thresholds)
import pprint
pprint.pprint(edges)

fig, ax = plot_transient(xp, yp, edges, thresholds=[0.1, 0.9], xscale=1, unit='s')
ticks = make_engineering_note_labels(unit='s', ticks=ax.get_xticks(), start=0)
ax.set_xticklabels(ticks[1])

ticks = make_engineering_note_labels(unit='V', ticks=ax.get_yticks(), start=0)
ax.set_yticklabels(ticks[1])

# fig = format_fig(fig)
plt.show()
pairs = transient_response.pair_edges(edges=edges, max_gap=None)
print(pairs, edges)

In [ ]:
import tdsxx4a
import python_utilities
from pathlib import Path

In [ ]:
d = Path("/home/simon/projects/ledsources/digital_pulser/V0.3.X/data/2025-05-12")
unit = "ns"

for fname in ["TEK00000.ISF", "TEK00001.ISF", "TEK00002.ISF", "TEK00003.ISF", "TEK00004.ISF", "TEK00005.ISF", "TEK00006.ISF", "TEK00007.ISF", "TEK00008.ISF", "TEK00009.ISF", "TEK00010.ISF"]:
    header, data = python_utilities.read_hp_spreadsheet(d / fname)
    #data = data[data['s']<=5e-6]
    y = data['Volts']
    x = np.array(data['s'])*1e9
    low, high, *_ = transient_response.detect_signal_levels(x, y, smooth_sigma=2, method="histogram")
    refererence_levels = [low, high]
    thresholds = transient_response.calculate_thresholds(x, y, refererence_levels)
    edges = detect_edges(x, y, thresholds)
    fig, ax = plot_transient(x, y, edges, thresholds=[0.1, 0.9], xscale=1e3, unit="ps")
    ax.plot(x, [min(refererence_levels)]*len(x), '--k')
    ax.plot(x, [max(refererence_levels)]*len(x), '--k')
    ax.set_xlabel("Time")
    ax.set_ylabel("Output")
    ticks = make_engineering_note_labels(unit=unit, ticks=ax.get_xticks(), start=0)
    ax.set_xticklabels(ticks[1])
    
    ticks = make_engineering_note_labels(unit='V', ticks=ax.get_yticks(), start=0)
    ax.set_yticklabels(ticks[1])
    fig.savefig(d / f"step-response-{fname}.svg")
    plt.show()